# TODO

* __Check with Minhchau on best method to establish neighbors__ (currently just counting links)
* __Export to json__ 
* __Create pipeline to run through all methods__
* __Add comments to make clear__




In [1]:
import pandas as pd
from collections import Counter
import json
import pymongo
import re
import time

## Function that returns list of neighbors as counter

In [2]:
#get a list of the neighbors a record has based on counts of occurrences
def getNeighbors(organization_id):
    neighbors = Counter()
    with open('graph/graph_root_ref.txt', 'r') as myfile:
        for line in myfile.readlines():
            rawText = line.strip().split('\t')
            if rawText[1] == organization_id:
                neighbors[rawText[2]] += 1
    return neighbors.most_common()

neighbors = getNeighbors('GB-CHC-1071886')
neighbors

[('GB-1', 2), ('GB-CHC-1071886', 1)]

## Write Function where based on list of neighbors returns imputed data

In [30]:
#Using test scores to start with
with open('test_scores.json', 'r') as myfile:
    records = json.loads(myfile.read().decode('utf-8-sig'))

#Helper function to find a record's id
#If it is None, that means the record does not appear in our list
def findRecord(org_id):
    for record in enumerate(records):
        if record[1]['organization_id'] == org_id:
            return record[0]

#Function to check if there are missing values in the record

def findMissing(num):
    missing_datas = [u'missing_data_activity_date',
 u'missing_data_budget',
 u'missing_data_currency',
 u'missing_data_description',
 u'missing_data_document_link',
 u'missing_data_language',
 u'missing_data_participating_org',
 u'missing_data_result',
 u'missing_data_title',
 u'missing_data_transaction']
    
    record = records[num] #get the specific record
    fields_to_fill = [] #holder to contain fields we will try to impute
    for mis in missing_datas: #check all missing fields
        if record[mis] != 0:
            fields_to_fill.append(mis)
    return fields_to_fill
            
        
#Get a list of ids in our records
ids = []
for record in records:
    ids.append(record['organization_id'])
ids = ids[:10] #keep only first 10 to make it easy   

t0 = time.time()
found = 0
for cur_id in ids: #for each id
    cur_record = findRecord(cur_id) #get the record
    missingFields = findMissing(cur_record) #get the missing fields for that record
    neighbors = getNeighbors(cur_id) #get a list of neighbors for each record
    for field in missingFields: #check each field that is missing
        for neighbor in neighbors: #check each neighbor, we want the one that is closest
            cur_neighbor = findRecord(neighbor[0]) #look for a record of the neighbor
            if cur_neighbor != None: #if the record exists proceed
                neighborMissing = findMissing(cur_neighbor) #get the fields that are missing for the neighbor
                if field not in neighborMissing: #if a neighbor is not missing the same field we can impute
                    print 'can impute ' + str(field[8:]) + ' for ' + str(cur_id) + ' from ' + str(neighbor[0])
                    break
    
            
    
#     for neighbor in neighbors:
#         if neighbor[0] in ids:
#             print "This record " + str(cur_id) + " is linked to " + str(neighbor)
print time.time() - t0
    
        
# record = '21032'
# number = findRecord(record) #21032
# if number != None:
#     fields = findMissing(number)
#     neighbors = getNeighbors(record)
#     imputeData(list_of_neighbors = neighbors)
# else:
#     print "This record does not appear in our scores data"

#imputeData(list_of_neighbors = neighbors)

# u'organization_id': u'GB-CHC-1098752'

#Primitive structure to impute the missing fields

# def imputeData(list_of_neighbors):
#     allNone = True #logical flag for if all neighbors are None and don't appear in our records
#     for neighbor in list_of_neighbors: #check each neighbor
#         found = findRecord(neighbor)
#         if found != None: #if a neighbor is not None proceed
#             allNone = False
#             cur_neighbor = records[found] #get neighbor's record
#             if cur_neighbor['missing_data_document_link'] == 0.0: #here would check if a missing value is present
#                 print "Found this"
#     if allNone == True:
#         print "Could not impute data"

can impute data_document_link for GB-CHC-1133342 from GB-CHC-1127488
can impute data_document_link for NL-KVK-51018586 from NL-KVK-27327087
can impute data_result for NL-KVK-51018586 from NL-KVK-27327087
can impute data_language for GB-CHC-1115109 from XM-DAC-7
3.88020896912


can impute data_document_link for NL-KVK-51018586 from NL-KVK-27327087
can impute data_result for NL-KVK-51018586 from NL-KVK-27327087
can impute data_language for GB-CHC-1115109 from XM-DAC-7
13.0038030148


/Library/Python/2.7/site-packages/ipykernel/__main__.py:9: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [9]:
stuff = 'missing_data_document_link'

stuff = stuff[8:]

print stuff

data_document_link


In [5]:
#records[0]

## Refactoring above to work in pymongo

In [3]:
conn = pymongo.MongoClient()
#conn.database_names() # iati, local

db = conn.iati
print db.collection_names()

[u'organizations', u'activities', u'cleaned_orgs_full', u'activities_metadata', u'transactions', u'organizations_metadata']


In [16]:
organizations = db.cleaned_orgs_full
test = organizations.find_one({'organization_id' : '21032'})
missing_data = test['missing_data']
missing_data

{u'@default-currency': 0.0,
 u'@xml:lang': 0.0,
 u'activity-date': 0.0,
 u'budget': 0.0,
 u'description': 0.0,
 u'document-link': 1.0,
 u'participating-org': 0.0,
 u'result': 1.0,
 u'title': 0.05785123966942149,
 u'transaction': 0.0}

In [29]:
#load data from json because easier
with open('test_scores.json', 'r') as myfile:
    records = json.loads(myfile.read().decode('utf-8-sig'))

#list that will hold ids for testing
ids = []
for record in records:
    ids.append(record['organization_id'])
ids = ids[:10] #keep only first 10 to make it easy   

#Don't need to a find record function because using mongo

#find and return a list of missing fields for a given id
def findMissing(org_id):
    record = organizations.find_one({'organization_id': org_id}) #get the specific record
    fields_to_fill = [] #holder to contain fields we will try to impute
    missing_data = record['missing_data']
    for missing in missing_data.keys():
        if missing_data[missing] != 0.0:
            fields_to_fill.append(missing)
    return fields_to_fill

t0 = time.time()


for cur_id in ids: #for each id
    missingFields = findMissing(cur_id) #get the missing fields for that record
    neighbors = getNeighbors(cur_id) #get a list of neighbors for each record
    for field in missingFields: #check each field that is missing
        for neighbor in neighbors: #check each neighbor, we want the one that is closest
            cur_neighbor = organizations.find_one({'organization_id' : neighbor[0]}) #look for a record of the neighbor
            if cur_neighbor != None: #if the record exists proceed 
                neighborMissing = findMissing(neighbor[0]) #get the fields that are missing for the neighbor
                if field not in neighborMissing: #if a neighbor is not missing the same field we can impute
                    print 'can impute ' + str(field) + ' for ' + str(cur_id) + ' from ' + str(neighbor[0])
                    break
    
print time.time() - t0

can impute document-link for NL-KVK-51018586 from NL-KVK-27327087
can impute result for NL-KVK-51018586 from NL-KVK-27327087
can impute @xml:lang for GB-CHC-1115109 from XM-DAC-7
16.4473400116


In [17]:
#load data from json because easier
with open('test_scores.json', 'r') as myfile:
    records = json.loads(myfile.read().decode('utf-8-sig'))

#list that will hold ids for testing
ids = []
for record in records:
    ids.append(record['organization_id'])
ids = ids[:10]

[u'title', u'document-link', u'result']

## Output the above to a json format

## Old stuff might be useful

In [9]:
scoreData = pd.read_csv('scores.csv')

In [14]:
scoreData.rename(columns = {'Unnamed: 0': 'Index'}, inplace = True)
#ata.rename(columns={'gdp':'log(gdp)'}, inplace=True)

In [19]:
scoreData.columns

Index([u'Unnamed: 0', u'_id', u'benford_compliance.budget',
       u'benford_compliance.transaction', u'doc_size.avg', u'doc_size.max',
       u'doc_size.min', u'missing_data.@default-currency',
       u'missing_data.@xml:lang', u'missing_data.activity-date',
       u'missing_data.budget', u'missing_data.description',
       u'missing_data.document-link', u'missing_data.participating-org',
       u'missing_data.result', u'missing_data.title',
       u'missing_data.transaction', u'organization_id', u'records',
       u'relative_size.budget', u'relative_size.description',
       u'relative_size.document-link', u'relative_size.participating-org',
       u'relative_size.result', u'relative_size.title',
       u'relative_size.transaction', u'title_description_similarity',
       u'completeness', u'compliance', u'distance', u'organization_id.score',
       u'utility', u'grade'],
      dtype='object')

In [17]:
missingData = scoreData[[u'_id', u'missing_data.@default-currency',
       u'missing_data.@xml:lang', u'missing_data.activity-date',
       u'missing_data.budget', u'missing_data.description',
       u'missing_data.document-link', u'missing_data.participating-org',
       u'missing_data.result', u'missing_data.title',
       u'missing_data.transaction', u'organization_id']]

In [19]:
missingData.head()

,_id,missing_data.@default-currency,missing_data.@xml:lang,missing_data.activity-date,missing_data.budget,missing_data.description,missing_data.document-link,missing_data.participating-org,missing_data.result,missing_data.title,missing_data.transaction,organization_id
0,5780510b370d935726735f83,0,0.000000,0,0.000000,0,1.000000,0,1.0,0,0,GB-CHC-1098752
1,5780511f370d935726735f84,0,0.000000,0,1.000000,0,1.000000,0,1.0,0,0,GB-CHC-1133342
2,57805132370d935726735f85,0,0.000000,0,0.000000,0,0.400000,0,0.6,0,0,NL-KVK-51018586
3,57805146370d935726735f86,0,0.333333,0,0.000000,0,0.333333,0,1.0,0,0,GB-CHC-1109789
4,5780515a370d935726735f87,0,0.000000,0,0.018834,0,0.000000,0,1.0,0,0,44000


In [19]:
scoreData = pd.read_csv('scores.csv')

list_of_neighbors = ['NL-KVK-41048542', 'GB-CHC-1071886', 'US-EIN-941191246'] #some list of IDs
list_of_fields = ['compliance', 'grade']
# for field in list_of_fields:
#     list_of_imputes = []
#     for neighbor in list_of_neighbors:
#         filt = scoreData[scoreData.organization_id == neighbor]
#         list_of_imputes.append(filt.get_value(filt.index[0], col = field))
#     counts = Counter(list_of_imputes)
#     print str(field) + ' imputed data ' + str(counts.most_common(1)[0][0])

def getImputedData(neighbors):
    list_of_fields = ['compliance', 'grade']
    for field in list_of_fields:
        list_of_imputes = []
        for neighbor in neighbors:
            filt = scoreData[scoreData.organization_id == neighbor]
            list_of_imputes.append(filt.get_value(filt.index[0], col = field))
        counts = Counter(list_of_imputes)
        print str(field) + ' imputed data ' + str(counts.most_common(1)[0][0])
        
getImputedData(list_of_neighbors)   

# filt = scoreData[scoreData.organization_id == 'GB-CHC-1098752']
# filt.get_value(filt.index[0], col = 'compliance')
# counts = Counter(list_of_imputes)
# print counts.most_common(1)[0][0]


compliance imputed data 100.0
grade imputed data C


## Pymongo Stuff

[u'organizations', u'activities', u'cleaned_orgs_full', u'activities_metadata', u'transactions', u'organizations_metadata']


In [30]:
for i in test['missing_data'].keys():
    if test['missing_data'][i] != 0:
        print i

title
document-link
result
